In [2]:
import numpy as np
import cv2
import json
import os
import argparse
import sys

In [36]:
example = True
example= example and example
example

True

In [34]:
True and (True and False)

False

In [179]:
class BoundingBox:
    '''
    Bounding box with corresponding parametes
    Params:
        center_x (float): x coordinate of the center
        center_y (float): y coordinate of the center
        width (float): width of the bounding box
        height (float): height of the bounding box
        label (str): label name
    '''
    def __init__(self, center_x: float, center_y: float, width: float, height: float, label: str):
        self.center_x = center_x
        self.center_y = center_y
        self.width = width
        self.height = height
        self.label = label
        
    def __eq__(self, other, threshold = 30):
        '''
        Equality method
        Args:
            self: object to compare
            other: object to compare
            threshhold (int, can be float): threshhold for float BoundingBox's params estimation
        '''
        label_condition = self.label == other.label     
        coords_condition = True
        coords_condition = coords_condition and abs(self.center_x - other.center_x)<threshold
        coords_condition = coords_condition and abs(self.center_y - other.center_y)<threshold
        coords_condition = coords_condition and abs(self.width - other.width)<threshold
        coords_condition = coords_condition and abs(self.height - other.height)<threshold
        return label_condition and coords_condition
    
    def __hash__(self):
#         TODO: rewrite hash for set, because for set it doesn't work. Instead eq function is used to compare sets
        return hash((self.center_x, self.center_y, self.width, self.width, self.label))
    
    def __str__(self):
        return f'BoundingBox({self.label}, [{self.center_x}, {self.center_y}], [{self.width}, {self.height}])'
    def __repr__(self):
        return f'BoundingBox({self.label}, [{self.center_x}, {self.center_y}], [{self.width}, {self.height}])'

In [180]:
def eq(bbs1, bbs2):
    '''
    Compares two sets of BoundingBox objects (dumb method)
    '''
    result = 0
    for bb1 in bbs1:
        for bb2 in bbs2:
            if bb1==bb2:
                result+=1
                break
    print(result, len(bbs1))
    return result == len(bbs1) and len(bbs1) == len(bbs2)

In [181]:
# def class_colors(names):
#     """
#     Create a dict with one random BGR color for each
#     class name
#     """
#     return {name: (
#         np.random.randint(0, 255),
#         np.random.randint(0, 255),
#         np.random.randint(0, 255)) for name in names}

# def compare_labels(json_path1, json_path2, colors = None, classes = None):
#     '''
#     Compare labels from json files
#     Args:
#         json_path1 (str): path to json file with labels
#         json_path2 (str): path to json file with labels
#         colors: list of RGB color for each bounding boxes class
#     Returns:
#         Comparison report
#     '''

#     # 1. Initialize variables with bounding boxes data
#     with open(json_path1) as f:
#         img_data1 = json.load(f)
#     with open(json_path2) as f:
#         img_data2 = json.load(f)
        
#     if colors is None:
#         colors = class_colors(classes)
    

#     # 2. Create two black images with corresponding bounding boxes
#     width, height = 2592, 1520 
#     img1 = 255 * np.ones((width, height, 3), dtype = np.uint8)
#     img2 = 255 * np.ones((width, height, 3), dtype = np.uint8)
#     img_list = [(img1, img_data1), (img2, img_data2)]
#     # loop in which  labels are added into corresponding image
#     for img, img_data in img_list:
#         shape_list = img_data['shapes']
#         for rect in shape_list:
#             label = rect['label']
#             left, top = list(map(lambda x: int(round(x)), (rect['points'][0])))
#             right, bottom = list(map(lambda x: int(round(x)), (rect['points'][1])))
#             cv2.rectangle(img, (left, top), (right, bottom), colors[label])
    
#     mse = ((img1-img2)**2).mean()
#     return mse


def get_xywh(rect):
    '''
    Gets x, y, width and height from x1, y1, x2, y2 coordinates
    '''
    x_list=[rect['points'][0][0], rect['points'][1][0]]
    y_list=[rect['points'][0][1], rect['points'][1][1]]
    x_max = max(x_list)
    x_min = min(x_list)
    y_max = max(y_list)
    y_min = min(y_list)

    width = x_max-x_min
    height = y_max-y_min
    center_x,center_y = x_min+width/2.0 , y_min+height/2.0

    return center_x, center_y, width, height

def compare_labels(json_path1, json_path2):
    '''
    Compares labels of two labeled images
    '''
    # 1. initialize variables with bounding boxes data
    with open(json_path1) as f:
        img_data1 = json.load(f)
    with open(json_path2) as f:
        img_data2 = json.load(f)
    
    # 2. Build two sets of BoundingBox objects    
    img_data_list = [img_data1, img_data2]
    bbs_list = []
    for img_data in img_data_list:
        shape_list = img_data['shapes']
#         bbs = {BoundingBox(*[coord for coords in rect['points'] for coord in coords], rect['label']) for rect in shape_list}
        bbs = {BoundingBox(*get_xywh(rect), rect['label']) for rect in shape_list}
        bbs_list.append(bbs)
    
    bbs1 = bbs_list[0]
    bbs2 = bbs_list[1]
    print(bbs1, len(bbs1))
    print(bbs2, len(bbs2))
    # 3. Compare two sets using eq function
    return eq(bbs1, bbs2)

In [186]:
json_path1 = r'D:/Projects/metal_classification/data/examples/00000042(1).json'
json_path2 = r'D:/Projects/metal_classification/data/examples/00000042(2).json'
compare_labels(json_path1, json_path2)

{BoundingBox(valid_metal, [2545.8333333333335, 916.6666666666667], [88.33333333333303, 63.333333333333485]), BoundingBox(grapple, [1693.3333333333335, 853.3333333333334], [346.66666666666674, 543.3333333333333]), BoundingBox(valid_metal, [575.0, 1090.8333333333335], [170.0, 95.0]), BoundingBox(valid_metal, [2336.666666666667, 1157.5], [93.33333333333348, 51.66666666666674]), BoundingBox(magnet, [1673.3333333333335, 1081.6666666666667], [273.33333333333326, 270.0]), BoundingBox(valid_metal, [335.83333333333337, 1197.5], [181.66666666666669, 115.0])} 6
{BoundingBox(valid_metal, [2545.8333333333335, 916.6666666666667], [88.33333333333303, 63.333333333333485]), BoundingBox(grapple, [1693.3333333333335, 853.3333333333334], [346.66666666666674, 543.3333333333333]), BoundingBox(valid_metal, [575.0, 1090.8333333333335], [170.0, 95.0]), BoundingBox(valid_metal, [2336.666666666667, 1157.5], [93.33333333333348, 51.66666666666674]), BoundingBox(magnet, [1673.3333333333335, 1081.6666666666667], [27

True